In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

In [ ]:
using LinearAlgebra
using QPControl
using QPControl.Trajectories
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using MathOptInterface
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
using OSQP
using QPWalkingControl
using PlanarConvexHulls
using Rotations
import GeometryTypes
using GeometryTypes: Vec, HyperRectangle, Point
BLAS.set_num_threads(1)

In [ ]:
urdf = AtlasRobot.urdfpath()
mechanism = parse_urdf(urdf, floating=true)

# foot contact information
foot_points = AtlasRobot.foot_contact_points(mechanism)
foot_collision_elements = CollisionElement[]
for (bodyid, points) in foot_points
    body = findbody(mechanism, bodyid)
    for point in points
        push!(foot_collision_elements, CollisionElement(body, point.frame, Point(point.v)))
    end
end
soleframes = AtlasRobot.add_sole_frames!(mechanism)
foot_polygons = make_foot_polygons(mechanism, soleframes, foot_points; num_extreme_points=4);

# environment contact information
# environment = CollisionElement[
#     CollisionElement(root_body(mechanism), root_frame(mechanism), HalfSpace(SVector(0., 0., 1.), 0.0))
# ]
ground_frame = CartesianFrame3D("ground")
frame_height = -1.0
ground_to_root = Transform3D(ground_frame, root_frame(mechanism), SVector(0., 0., frame_height))
add_frame!(root_body(mechanism), ground_to_root)
robot_height_delta = -0.5
plane_height = -frame_height + robot_height_delta
dx, dy, dz = 5.0, 1.0, 2.0
ground = HyperRectangle(Vec(-1.0, -dy/2, plane_height - dz), Vec(dx, dy, dz))
environment = CollisionElement[
    CollisionElement(root_body(mechanism), ground_frame, ground)
]

# body lookup
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis");

# contact model
contact_model = ContactModel()
push!(contact_model, foot_collision_elements)
push!(contact_model, environment)
normal_model = hunt_crossley_hertz(; k=500e3)
# k_tangential = 20e3
# b_tangential = 2 * sqrt(k_tangential * mass(mechanism) / 10)
# tangential_model = ViscoelasticCoulombModel(0.8, k_tangential, b_tangential)
tangential_model = ViscoelasticCoulombModel(0.8, 20e3, 100.)
contact_force_model = SplitContactForceModel(normal_model, tangential_model)
set_contact_force_model!(contact_model, foot_collision_elements, environment, contact_force_model);

In [ ]:
# create low level controller
lowlevel = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    lowlevel = MomentumBasedController{4}(mechanism, optimizer,
        floatingjoint = findjoint(mechanism, "pelvis_to_world"));
    for (bodyid, points) in foot_points
        body = findbody(mechanism, bodyid)
        for point in points
            normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
            μ = contact_force_model.tangential.μ
            contact = addcontact!(lowlevel, body, point, normal, μ)
            contact.maxnormalforce[] = 1e6 # TODO
            contact.weight[] = 1e-3
        end
    end
    lowlevel
end;

In [ ]:
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)

In [ ]:
floating_joint = first(joints(mechanism))
configuration(nominalstate, floating_joint)[end] += robot_height_delta;

In [ ]:
# ICP stuff
zdes = center_of_mass(nominalstate).v[3] - 0.05
gz = norm(mechanism.gravitational_acceleration)
ω = sqrt(gz / zdes)
icptraj = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-6)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-8)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 10000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    max_polygon_sides = 6 # TODO
    num_segments = 15
    ICPTrajectoryGenerator{Float64, max_polygon_sides}(optimizer, num_segments, ω)
end
linear_momentum_controller = ICPController(mechanism, icptraj, zdes);

In [ ]:
# walking state machine
statemachine = let
    contacts = Dict(BodyID(body) => contact for (body, contact) in lowlevel.contacts)
    ICPWalkingStateMachine(mechanism, contacts, icptraj)
end;

In [ ]:
QPWalkingControl.init_footstep_plan!(statemachine, nominalstate, foot_polygons);

In [ ]:
# create high level controller
controller = HumanoidQPController(lowlevel, pelvis, nominalstate, 
    statemachine, collect(values(statemachine.end_effector_controllers)), linear_momentum_controller);

In [ ]:
# create visualizer
using MeshCat
using MeshCatMechanisms
using MechanismGeometries

if !(@isdefined gui) || !any(isopen, RigidBodySim.Visualization.mechanism_visualizer(gui.visualizer).visualizer.core.scope.pool.connections)
    pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0))
    link_colors = Dict(map(foot -> string(foot) => RGBA(0.7f0, 0.7f0, 0.7f0, 0.3f0), feet))
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path=[AtlasRobot.packagepath()], link_colors=link_colors)
    vis = Visualizer()
    mvis = MechanismVisualizer(mechanism, visuals, vis)
    setelement!(mvis, ground_frame, ground)
    pvis = PushRecoveryVisualizer(mvis)
    gui = GUI(pvis, usernode=Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    open(gui)
end
copyto!(gui.visualizer, nominalstate);

In [ ]:
# create ODEProblem
state = MechanismState(mechanism)
copyto!(state, nominalstate)
Δt = 1 / 500
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
damping = JointDamping{Float64}(mechanism, AtlasRobot.urdfpath())
dynamics = Dynamics(
    mechanism, 
    SumController(similar(velocity(state)), (pcontroller, damping, pushapplier));
    contact_model=contact_model)
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui; max_fps=60))
# callback = CallbackSet(gui; max_fps=30)
tspan = (0., 18.)
contact_state = SoftContactState(contact_model)
problem = ODEProblem(dynamics, (state, contact_state), tspan)#; callback=callback)

In [ ]:
# using BenchmarkTools
# τ = similar(velocity(nominalstate))
# @btime $controller($τ, 0.0, $nominalstate) # 90 allocations: 2.98 KiB

In [ ]:
# simulate
QPWalkingControl.init_footstep_plan!(statemachine, nominalstate, foot_polygons);
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)#, dtmax=1e-3);
last(sol.t)

In [ ]:
setanimation!(vis, Animation(mvis, sol))

In [ ]:
# using Test
# copyto!(state, last(sol.u))
# @test last(sol.t) == last(tspan)
# @test center_of_mass(state).v[1] > 1.5
# @test center_of_mass(state).v[3] > 0.7

In [ ]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult